In [2]:
# -----Import packages
import xarray as xr
import os
import numpy as np
import glob
from matplotlib import pyplot as plt, dates
import matplotlib
import rasterio as rio
import geopandas as gpd
import pandas as pd
import sys
import ee
import geedim as gd
import json
from tqdm.auto import tqdm
from joblib import dump, load
from shapely.geometry import MultiPolygon, Polygon

NameError: name 'base_path' is not defined

In [20]:
# path to snow-cover-mapping/ - Make sure you include a "/" at the end
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping/'

# add path to functions
sys.path.insert(1, base_path+'functions/')
import pipeline_utils as f

study_sites_path = '/Users/raineyaberle/Google Drive/My Drive/Research/CryoGARS-Glaciology/Advising/student-research/Alexandra-Friel/snow_cover_mapping_application/study-sites/'
site_names =[x for x in sorted(os.listdir(study_sites_path)) if (not x.startswith('.')) and ('csv' not in x)]
bgotus_site_names = ['Wolverine', 'Gulkana', 'LemonCreek', 'SouthCascade', 'Sperry']
site_names = [x for x in site_names if x not in bgotus_site_names]
site_names

['Blue',
 'Boulder',
 'Carbon',
 'Coleman',
 'Easton',
 'Emmons',
 'Hidden',
 'Hoh',
 'RGI60-01.00037',
 'RGI60-01.00038',
 'RGI60-01.00046',
 'RGI60-01.00312',
 'RGI60-01.00566',
 'RGI60-01.00576',
 'RGI60-01.01151',
 'RGI60-01.01390',
 'RGI60-01.01524',
 'RGI60-01.01733',
 'RGI60-01.03861',
 'RGI60-01.04375',
 'RGI60-01.06268',
 'RGI60-01.06722',
 'RGI60-01.08155',
 'RGI60-01.08174',
 'RGI60-01.08246',
 'RGI60-01.08248',
 'RGI60-01.08262',
 'RGI60-01.08288',
 'RGI60-01.08296',
 'RGI60-01.08302',
 'RGI60-01.08336',
 'RGI60-01.08353',
 'RGI60-01.08389',
 'RGI60-01.08395',
 'RGI60-01.08403',
 'RGI60-01.08412',
 'RGI60-01.08427',
 'RGI60-01.09148',
 'RGI60-01.09216',
 'RGI60-01.09411',
 'RGI60-01.09639',
 'RGI60-01.10196',
 'RGI60-01.10555',
 'RGI60-01.10689',
 'RGI60-01.10778',
 'RGI60-01.10851',
 'RGI60-01.10857',
 'RGI60-01.11616',
 'RGI60-01.11654',
 'RGI60-01.11788',
 'RGI60-01.12370',
 'RGI60-01.12425',
 'RGI60-01.12635',
 'RGI60-01.13696',
 'RGI60-01.14391',
 'RGI60-01.14443',
 'R

In [17]:
ee.Initialize()

In [28]:
for site_name in tqdm(site_names[30+23+9:]):
    
    print(site_name)
    
    # check if snowlines have been delineated yet
    if len(glob.glob(study_sites_path + site_name + '/imagery/snowlines/*.csv'))<1:
        print('No snowlines yet, continuing...')
        print(' ')
        continue
    
    # path to folder containing AOI files
    AOI_path = '/Users/raineyaberle/Google Drive/My Drive/Research/CryoGARS-Glaciology/Advising/student-research/Alexandra-Friel/snow_cover_mapping_application/study-sites/' + site_name + '/AOIs/'
    # AOI file name
    AOI_fn = os.path.basename(glob.glob(AOI_path + site_name + '*_outline.shp')[0])
    # path to folder containing DEM raster file
    # Note: set DEM_fn=None if you want to use the ArcticDEM or NASADEM via Google Earth Engine
    DEM_path = AOI_path + '../DEMs/'
    # DEM file name
    DEM_fn = None 
    
    if len(glob.glob(DEM_path + '*ArcticDEM*.tif')) > 0:
        [os.remove(x) for x in glob.glob(DEM_path + '*ArcticDEM*.tif')]
    elif len(glob.glob(DEM_path + '*NASADEM*.tif')) > 0:
        [os.remove(x) for x in glob.glob(DEM_path + '*NASADEM*.tif')]
    print('removed old DEM file')
        
    # -----Load AOI as gpd.GeoDataFrame
    AOI = gpd.read_file(AOI_path + AOI_fn)
    # reproject the AOI to WGS to solve for the optimal UTM zone
    AOI_WGS = AOI.to_crs('EPSG:4326')
    AOI_WGS_centroid = [AOI_WGS.geometry[0].centroid.xy[0][0],
                        AOI_WGS.geometry[0].centroid.xy[1][0]]
    # grab the optimal UTM zone EPSG code
    epsg_UTM = f.convert_wgs_to_utm(AOI_WGS_centroid[0], AOI_WGS_centroid[1])
    print('Optimal UTM CRS = EPSG:' + str(epsg_UTM))
    # reproject AOI to the optimal UTM zone
    AOI_UTM = AOI.to_crs('EPSG:'+epsg_UTM)

    # -----Load DEM as Xarray DataSet
    if DEM_fn is None:
        # query GEE for DEM
        DEM = f.query_gee_for_dem(AOI_UTM, base_path, site_name, DEM_path)
    else:
        # load DEM as xarray DataSet
        DEM = xr.open_dataset(DEM_path + DEM_fn)
        DEM = DEM.rename({'band_data': 'elevation'})
        # reproject the DEM to the optimal UTM zone
        DEM = DEM.rio.reproject('EPSG:'+str(epsg_UTM)).rio.write_crs('EPSG:'+str(epsg_UTM))
        # set no data values to NaN
        DEM = xr.where((DEM > 1e38) | (DEM <= -9999), np.nan, DEM)
        
    print(' ')

  0%|          | 0/81 [00:00<?, ?it/s]

RGI60-01.17183
removed old DEM file
Optimal UTM CRS = EPSG:32607
ArcticDEM coverage over AOI


Consider adjusting `region`, `scale` and/or `dtype` to reduce the RGI60-01.17183_ArcticDEM_clip.tif download size (raw: 11.73 GB).


RGI60-01.17183_ArcticDEM_clip.tif: | | 0.00/11.7G (raw) [  0.0%] in 00

Transforming elevations from the ellipsoid to the geoid...
DEM re-saved with elevations referenced to the EGM96 geoid.
 
RGI60-01.17348
No snowlines yet, continuing...
 
RGI60-01.17423
No snowlines yet, continuing...
 
RGI60-01.17464
removed old DEM file
Optimal UTM CRS = EPSG:32607
ArcticDEM coverage over AOI


...

RGI60-02.17738_NASADEM_clip.tif: | | 0.00/454k (raw) [  0.0%] in 00:00

 
SitKusa
removed old DEM file
Optimal UTM CRS = EPSG:32607
ArcticDEM coverage over AOI


Consider adjusting `region`, `scale` and/or `dtype` to reduce the SitKusa_ArcticDEM_clip.tif download size (raw: 2.70 GB).


SitKusa_ArcticDEM_clip.tif: | | 0.00/2.70G (raw) [  0.0%] in 00:00 (et

Transforming elevations from the ellipsoid to the geoid...
DEM re-saved with elevations referenced to the EGM96 geoid.
 
Tahoma
No snowlines yet, continuing...
 
Variegated
No snowlines yet, continuing...
 
